In [1]:
!pip install ultralytics

^C


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 1.2 MB/s eta 0:00:01
   -------------------- ------------------- 0.5/1.0 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 1.2 MB/s  0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 1.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.6 MB 1.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.6 MB 1.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.6 MB 1.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.6 MB 1.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.

  ERROR: Failed to restore c:\users\mayur\appdata\local\programs\python\python313\lib\site-packages\torch\
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\Users\\mayur\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\torch-2.8.0.dist-info\\INSTALLERawp9vpoj.tmp'



In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import cv2
from ultralytics import YOLO
import numpy as np
import base64
import io
from PIL import Image

ModuleNotFoundError: No module named 'google.colab'

In [9]:
js_code = '''
function initCamera() {
    return new Promise((resolve, reject) => {
        const video = document.createElement('video');
        video.style.display = 'none';
        document.body.appendChild(video);
        const streamPromise = navigator.mediaDevices.getUserMedia({video: true});
        streamPromise.then((stream) => {
            video.srcObject = stream;
            video.onloadedmetadata = () => {
                resolve(video);
            };
            video.play();
        }).catch((error) => {
            reject(error);
        });
    });
}

async function takePhoto() {
    const video = await initCamera();
    const canvas = document.createElement('canvas');
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    const context = canvas.getContext('2d');
    context.drawImage(video, 0, 0, canvas.width, canvas.height);
    const img = canvas.toDataURL('image/jpeg');
    return img;
}
'''
display(Javascript(js_code))


<IPython.core.display.Javascript object>

In [5]:
def js_to_image(js_reply):
  image_bytes = base64.b64decode(js_reply.split(',')[1])
  image_PIL = Image.open(io.BytesIO(image_bytes))
  image_np = np.array(image_PIL)
  frame = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
  return frame

In [6]:
yolo = YOLO('yolov8s.pt')

In [7]:
def getColours(cls_num):
    base_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    color_index = cls_num % len(base_colors)
    increments = [(1, -2, 1), (-2, 1, -1), (1, -1, 2)]
    color = [base_colors[color_index][i] + increments[color_index][i] *
    (cls_num // len(base_colors)) % 256 for i in range(3)]
    return tuple(color)

In [10]:
while True:
    js_reply = eval_js('takePhoto()')  # Take webcam snapshot
    frame = js_to_image(js_reply)      # Convert to OpenCV

    if frame is None:
        continue

    results = yolo.track(frame, stream=True)  # Detect & track objects

    for result in results:
        classes_names = result.names  # Class name dictionary

        for box in result.boxes:
            if box.conf[0] > 0.4:  # Only if confidence > 40%
                [x1, y1, x2, y2] = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                cls = int(box.cls[0])  # Object class ID
                class_name = classes_names[cls]
                colour = getColours(cls)

                cv2.rectangle(frame, (x1, y1), (x2, y2), colour, 2)  # Draw box
                cv2.putText(frame, f'{class_name} {box.conf[0]:.2f}', (x1, y1),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, colour, 2)

    cv2_imshow(frame)  # Show the image

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Stop if 'q' pressed
        break

cv2.destroyAllWindows()

MessageError: ReferenceError: takePhoto is not defined